In [4]:
from bs4 import BeautifulSoup
import requests
import openai
import os

openai.api_key = os.getenv("OPENAI_API_KEY")

# Retrieve the HTML content
url = 'https://sf.funcheap.com/events/san-francisco/'
response = requests.get(url)
html = response.content

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Find the table element
table = soup.find('table')

# Find all the links within the table
links = table.find_all('a')

# Extract the href attributes from each link
hrefs = [link.get('href') for link in links]
headlines = [link.text for link in links]

In [71]:
def generate_prompt(event):
    prompt = """Extract information from the event below and store it into a JSON with the following structure {venue: venue_name, date: date/time, age: age_requirement, price: price, location: address}""" + ' and enclose each key and value in the JSON structure with double quotes.' + '\n' + "Event: " + event
    return prompt

def generate_desc(event):
    prompt = 'Summarize the following event description below in a sentence or two to find the most important aspects that explain what the event is about.' + '\n' + 'Event Description: ' + event
    return prompt

In [79]:
import json 
import pandas as pd

sf_events = pd.DataFrame()

for link,headline in zip(hrefs,headlines):
  headline = headline.strip()
  if link is not None and len(headline) > 0: 
    response = requests.get(link)
    html = response.content
    soup = BeautifulSoup(html, 'html.parser')
    p_tags = soup.find_all('p')
    event_name = soup.find('h1')
    event_name = event_name.get_text()
    text = ''
    div = soup.find('div', {'id': 'stats'})
    text += div.get_text()
    
    response = openai.Completion.create(
    model="text-davinci-003",
    prompt=generate_prompt(text),
    temperature=0.0,
    max_tokens = 200,
    )

    response_text = response['choices'][0]['text'].strip()
    try:
      response_dict = json.loads(response_text)
    except Exception as e:
      print(str(e))
      print(response_text)

    description = ''
    for p in p_tags:
      description += p.get_text()
    
    
    event_response = openai.Completion.create(
    model="text-curie-001",
    prompt=generate_desc(description),
    temperature=0.3,
    max_tokens = 500,
    timeout=0
    )

    event_text = event_response['choices'][0]['text'].strip()
    response_dict['event_description'] = event_text
    response_dict['event_name'] = event_name

    sf_events = sf_events.append(response_dict, ignore_index=True)
    
    


In [82]:
sf_events

,venue,date,age,price,location,event_description,event_name
0,Asian Art Museum,"Thursday, February 16, 2023 - 5:00 pm to 8:00 pm",13-17,"$10 for adults, $8.50 for seniors, students an...","200 Larkin Street San Francisco, CA Civic Cent...",,Asian Art Museum’s $10 Thursday Nights (SF)
1,Asian Art Museum,"Thursday, February 16, 2023 - 5:00 pm to 8:00 pm",13-17,"$10 for adults, $8.50 for seniors, students an...","200 Larkin Street San Francisco, CA Civic Cent...",The Asian Art Museum is open until 8 pm on Thu...,Asian Art Museum’s $10 Thursday Nights (SF)
2,Museum of Craft and Design,"Thursday, February 16, 2023 - 5:30 pm to 9:00 pm",,$8,"2569 Third Street, San Francisco, CA 94107Dogp...",The event is about making things in the dark.,“Glow in the Dark” After Hours at SF’s Museum ...
3,Del Mar,"Thursday, February 16, 2023 - 7:00 pm to 2:00 am",No cover all night,Free,"2125 Lombard St, San Francisco, CA 94123Cow Ho...",,“Throwback Thursday” at Del Mar: ’90s to ’00s ...
4,Grace Cathedral,"Thursday, February 16, 2023 - 7:30 pm to 9:30 pm",General Admission,USD 25.00,"1100 California Street. San Francisco, CANob H...",,“That Week with the Bachs” New Musical at Grac...
...,...,...,...,...,...,...,...
113,Fish and Farm,"Friday, March 3, 2023 - 7:00 pm",Free / RSVP,FREE,"424 Clay St, San Francisco, CAFinancial Distri...",The event is a comedy show featuring up-and-co...,“Deep Cut Comedy” Live Stand Up at Fish & Farm...
114,Secret Location (SF),"Friday, March 3, 2023 - 8:00 pm to 10:00 pm",,FREE,"San Francisco, CA",The San Francisco Bike Party is a monthly cele...,San Francisco Bike Party (First Fridays)
115,Neck of The Woods,"Saturday, March 4, 2023 - 7:00 pm to 9:00 pm",Free / RSVP,Donations ($5-$10) greatly appreciated. $10 ad...,"406 Clement Street, San Francisco, CA Richmond...",This event is a comedy showcase where Bay Area...,“Crazy Funny Asians” Comedy Showcase (SF)
116,Neck of The Woods,"Saturday, March 4, 2023 - 7:00 pm to 9:00 pm",Free / RSVP,Donations ($5-$10) greatly appreciated. $10 ad...,"406 Clement Street, San Francisco, CA Richmond...",The Crazy Funny Asians comedy showcase is a fr...,“Crazy Funny Asians” Comedy Showcase (SF)


In [89]:
from opencage.geocoder import OpenCageGeocode
from shapely.geometry import Point

key = '21b734acde524209895d1c01fc8e6eea'
geocoder = OpenCageGeocode(key)
locations = []

for index,row in sf_events.iterrows():
    v = row['location']
    results = geocoder.geocode(v)
    loc = Point(results[0]['geometry']['lat'],results[0]['geometry']['lng'])
    locations.append(loc)
    


In [92]:
sf_events['geolocation'] = locations

/Users/gauravmohan/opt/anaconda3/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [94]:
sf_events.to_csv('funcheap_events.csv')